In [1]:
import re
import pandas as pd
import numpy as np
from google.colab import drive
import os
import itertools
from concurrent.futures import ThreadPoolExecutor, as_completed
#drive.mount('/content/drive')

In [2]:
def model_w_parameters(alpha, beta, gamma, data):
    """This is a code that takes alpha, beta, and gamma parameters as inputs.
       Applies exponential smoothing with damped trend method on them.
       Finally gives one-step-ahead results for given data.
       levels, takes first element of the data list as inputs.
       slopes, takes average trend of the data."""
    observation = data
    levels = [data[0]]
    slopes = [(data[0]-data[len(data)-1])/ len(data)]
    forecasts = []

    for i, o in enumerate(observation, start=0):
        level_value = o*(alpha) + (gamma*slopes[i] + levels[i])*(1-alpha)
        levels.append(level_value)
        slopes.append(((levels[i + 1] - levels[i])*beta + (1 - beta) * gamma*slopes[i]))
        forecasts.append(levels[i] + gamma*slopes[i])

    return forecasts, slopes[-1]

def calculate_mape(params, data):
    """Calculate MAPE for given parameters and data"""
    alpha, beta, gamma = params
    forecasts = model_w_parameters(alpha, beta, gamma, data)[0]
    return ((np.abs(np.array(data) - np.array(forecasts))/np.array(data))*100).mean(), params

def grid_search(param_grid, data):
    """Perform grid search over parameter space"""
    results = []

    # Generate all parameter combinations
    all_params = list(itertools.product(*param_grid))

    # Paralel işlem yapsın  diye var kütüphaneyi araştırmanızı öneririm. Keyifli bi kütüphane ThreadPoolExecutor. Hızımızı temelde bu arttırıyor.
    if len(all_params) < 1000:
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = [executor.submit(calculate_mape, params, data) for params in all_params]
            for future in as_completed(futures):
                mape, params = future.result()
                results.append((mape, params))
    else:
        # For larger spaces, batch processing is more efficient
        batch_size = 1000
        for i in range(0, len(all_params), batch_size):
            batch = all_params[i:i+batch_size]
            for params in batch:
                mape, _ = calculate_mape(params, data)
                results.append((mape, params))

    # Find best parameters
    best_mape, best_params = min(results, key=lambda x: x[0])
    return best_mape, best_params

def optimizer(data):
    """
    This code takes only the data given.
    By using an efficient grid search, it finds optimal alpha, beta, and gamma parameters.
    Data is a list input.
    Outputs:
    - DataFrame with two columns: Observation and Forecasts
    - Slope value from the model
    - Optimal alpha, beta, gamma parameters
    - MAPE (Mean Absolute Percentage Error)
    """
    # First coarse grid search
    param_grid = [
        np.linspace(0.01, 0.99, 20),  # alpha
        np.linspace(0.01, 0.99, 20),  # beta
        np.linspace(0.80, 0.98, 10)   # gamma değerinin alt sınırını 0.01 yaparak değişkenin restrictionını ortadan kaldırabiliriz.
    ]

    _, (alpha, beta, gamma) = grid_search(param_grid, data)

    # Refine search around best parameters
    alpha_range = np.linspace(max(0.001, alpha-0.05), min(0.999, alpha+0.05), 10)
    beta_range = np.linspace(max(0.001, beta-0.05), min(0.999, beta+0.05), 10)
    gamma_range = np.linspace(max(0.8, gamma-0.02), min(0.98, gamma+0.02), 10) #yine aynı şekilde 0.8'i 0.001 yapmamız gerekiyor.

    param_grid = [alpha_range, beta_range, gamma_range]
    _, (alpha, beta, gamma) = grid_search(param_grid, data)

    # Final fine-tuning
    alpha_range = np.linspace(max(0.0001, alpha-0.01), min(0.9999, alpha+0.01), 10)
    beta_range = np.linspace(max(0.0001, beta-0.01), min(0.9999, beta+0.01), 10)
    gamma_range = np.linspace(max(0.8, gamma-0.01), min(0.98, gamma+0.01), 10) #son olarak burada.

    param_grid = [alpha_range, beta_range, gamma_range]
    mape, (alpha, beta, gamma) = grid_search(param_grid, data)

    # Generate final forecasts
    forecasts, slope = model_w_parameters(alpha, beta, gamma, data)

    return pd.DataFrame({'Forecast': forecasts, 'Actual_Observation': data}), slope, alpha, beta, gamma, mape
def data_capturer(path):
  """ This code is extracting_data needed from the torbali_veriler.xlsx file"""
  df = pd.read_excel(path, sheet_name=0, usecols="BE:BW", skiprows=8, nrows=45)
  rowTags = pd.read_excel(path , sheet_name=0 , usecols='A' , skiprows = 8 , nrows = 45)
  df = pd.concat([rowTags , df] , axis = 1)
  dictoflist = {}
  for i , r in df.iterrows():
    dictoflist[r[0]]= r[1:]
  df = pd.DataFrame(dictoflist).reset_index()
  dftrain = df.iloc[:14,:]
  dftest = df.iloc[14:,:]
  return df , dftrain , dftest

def optimizing_parameter(train_data):
  columns = train_data.columns[1:]
  model_data = [ optimizer(train_data[e]) for e in columns ]
  slopes = [t[1] for t in model_data]
  alphas = [t[2] for t in model_data]
  betas = [t[3] for t in model_data]
  gammas = [t[4] for t in model_data]
  train_mapes = [t[5] for t in model_data]
  return pd.DataFrame({'ZoneId' : columns ,  'trend':slopes , 'alpha':alphas , 'beta'  :betas , 'gamma': gammas , 'train_mape' : train_mapes})

def forecaster(test_data ,train_data , optimizing_parameters , step = 2 , reason = 'test'):
  columns = optimizing_parameters['ZoneId']
  slopes = optimizing_parameters['trend']
  gammas = optimizing_parameters['gamma']
  dftest = test_data
  dftrain = train_data
  h = step

  if reason == 'test':
    h = len(dftest)
    d_values = [np.array([gammas[i]**k for k in range(1, h + 1) ]) for i in range(len(columns))]
    forecasts = [[dftest[d].iloc[0] + slopes[i] * (d_values[i][:p]).sum() for p in range(1,h+1)] for i,d in enumerate(columns)]
  else:
    d_values = [np.array([gammas[i]**k for k in range(1, h + 1) ]) for i in range(len(columns))]
    forecasts = [[dftest[d].iloc[-1] + slopes[i] * (d_values[i][:p]).sum() for p in range(1,h+1)] for i,d in enumerate(columns)]
  return forecasts
def excel_writer(izmir_poligon_path , ilce):
  dftrain = pd.read_excel(izmir_poligon_path +'/' +ilce + '/' + ilce +  '_meandata-50.xlsx').iloc[:14,1:]
  dftest = pd.read_excel(izmir_poligon_path + '/' +ilce + '/' + ilce +  '_meandata-50.xlsx').iloc[14:,1:]
  df = pd.read_excel(izmir_poligon_path +'/' +ilce + '/' + ilce +  '_meandata-50.xlsx').iloc[:,1:]
  parameter_df = optimizing_parameter(dftrain)
  forecastsList = forecaster(dftest,dftrain,parameter_df,reason='test')
  mape_list = []
  mae_list = []
  mse_list = []
  for i,e in enumerate(dftest.columns[1:]) :
    forecasted_value_column = e+'_forecasted'
    error_column = e + '_error_term'
    dftest[forecasted_value_column] = forecastsList[i]
    dftest[error_column] = dftest[e] - dftest[forecasted_value_column]
    mape_list.append(((abs(dftest[error_column]/dftest[e]))*100).mean())
    mae_list.append(abs(dftest[error_column]).mean())
    mse_list.append((dftest[error_column]**2).mean())
  error_df = pd.DataFrame({'ZoneId': dftrain.columns[1:] , 'testMAPE':mape_list,'testMSE':mse_list,'testMAE':mae_list})
  final_df = parameter_df.merge(error_df , on = 'ZoneId')
  final_df.to_excel(izmir_poligon_path + '/' + ilce + '/' +ilce+'_errors_params-50.xlsx')

In [3]:
izmir_poligon_path = '/content'
ilce = 'all'
excel_writer(izmir_poligon_path , ilce)# yalnızca üstteki iki değişken değişmesi durumunda kod çalışacaktır ve error excelini kaydedecektir.

Görüntülenen çıkış son 5000 satıra kısaltıldı.
<ipython-input-2-c04316301350>:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftest[forecasted_value_column] = forecastsList[i]
<ipython-input-2-c04316301350>:145: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftest[error_column] = dftest[e] - dftest[forecasted_value_column]
<ipython-input-2-c04316301350>:144: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c